<h5 style="color:green"> All Models' features need to be 2D Arrays even if there is one feature </h3>
<ul>
<li> No documnetation
<li> Better OOP Design and Redundancy Omitting
<li> To be implemented :
    <ul>
        <li> Gaussian Naive Bayes
        <li> Logistic Regression
        <li> Support Vector Machine
        <li> Gradient Boosting
        <li> DBSCAN and HDBSCAN
        <li> K-Means
        <li> PCA - UMAP
        <li> Reinforcement Learning
        <li> AlphaZero
        <li> Factorization Methods
        <li> Convolutional Neural Networks
        <li> RNN + LSTM
        <li> Transformers
    </ul>
<li> Needs Better Implementations :
    <ul>
        <li> Faster BallTree / KDTree Algorithms for KNN
    </ul>
</ul>

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier as SklearnGBC
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier as SklearnAdaBoost
from sklearn.impute import SimpleImputer as SklearnSimpleImputer
from sklearn.impute import KNNImputer as SklearnKNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer as SklearnIterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score as sk_cross_val_score
from sklearn.model_selection import GridSearchCV as SklearnGridSearchCV

import decision_tree
import random_forest
import linear_regression
import naive_bayes
import nearest_neighbors
from gradient_boosting import GradientBoostingClassifier
from adaboost import AdaBoostClassifier
from imputation import SimpleImputer
from imputation import KNNImputer
from imputation import IterativeImputer
from kfold import KFold
from grid_search import GridSearchCV

from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pickle

import time
import matplotlib.pyplot as plt

In [2]:
def load_dataset(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def compareModels(models, X, y , metric):
    for i, model in enumerate(models):
        print(f"Model {i}")
        start = time.perf_counter()
        model.fit(X[0], y[0])
        end = time.perf_counter()
        print(f"Time to fit : {end - start} s")
        print(f"Performance : {metric(model.predict(X[1]), y[1])}")


In [4]:
df = load_dataset("citiesSmall.pkl")
X = df['X']
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)
X_train.shape

(300, 2)

In [11]:
depth = [i for i in range(1, 24)]
accuracy = [[], []]
execution_time = [[], []]

for i in depth:
    print("==========================================")
    models = []
    models.append(DecisionTreeClassifier(criterion='gini', max_depth=i))
    models.append(decision_tree.DecisionTreeClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)
    

Performance : 0.85
Model 0
Time to fit : 0.0006879000011394965 s
Performance : 0.86
Model 1
Time to fit : 0.10280260000035923 s
Performance : 0.85
Model 0
Time to fit : 0.0004459000010683667 s
Performance : 0.86
Model 1
Time to fit : 0.10439910000059172 s
Performance : 0.85
Model 0
Time to fit : 0.0006955999997444451 s
Performance : 0.85
Model 1
Time to fit : 0.10723030000008293 s
Performance : 0.85
Model 0
Time to fit : 0.0005154000009497395 s
Performance : 0.85
Model 1
Time to fit : 0.10501679999833868 s
Performance : 0.85
Model 0
Time to fit : 0.0005252999999356689 s
Performance : 0.85
Model 1
Time to fit : 0.11209600000074715 s
Performance : 0.85
Model 0
Time to fit : 0.0006604000009247102 s
Performance : 0.84
Model 1
Time to fit : 0.1041851999998471 s
Performance : 0.85
Model 0
Time to fit : 0.0006352000000333646 s
Performance : 0.86
Model 1
Time to fit : 0.10608929999943939 s
Performance : 0.85


In [12]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

depth = [i for i in range(1, 24)]
accuracy = [[], []]
execution_time = [[], []]

for i in depth:
    print(f"Depth {i} ==========================================")
    models = []
    models.append(RandomForestClassifier(criterion='gini', max_depth=i))
    models.append(random_forest.RandomForestClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

Depth 1 ==========================================
Model 0
Time to fit : 0.060916399999769055 s
Performance : 0.7
Model 1
Time to fit : 0.09508350000032806 s


In [ ]:
X, y = datasets.make_regression(n_samples=10000, n_features=10, noise=1, bias=19, random_state=4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

models = [LinearRegression(), linear_regression.LinearRegression()]
compareModels(models, (X_train, X_test), (y_train, y_test), mean_squared_error)

# fig = plt.figure(figsize=(10, 8))
# plt.scatter(X, y, marker='o')
# plt.plot(X, model.predict(X), marker = 'o')
# plt.show()

In [ ]:
X, y = datasets.make_classification(n_samples=18000, n_features=5, n_informative=5, n_redundant=0, n_classes=2, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

models = [MultinomialNB(), naive_bayes.NaiveBayes()]
compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

In [ ]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

for k in range(5, 101):
    print(f"\n{k} neighbors ============================\n")
    models = [KNeighborsClassifier(n_neighbors=k), nearest_neighbors.KNN(k)]
    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

In [ ]:
from deep_learning import *
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = Module()

# Add layers to your MLP model
mlp.add_layer(Layer(input_dim=X_train.shape[1], output_dim=64))  # Adjust the dimensions as needed
mlp.add_layer(Sigmoid(64, 64))  # Add activation layers and adjust dimensions as needed
mlp.add_layer(Layer(64, 1))

# Fit the model to the training data
mlp.fit(X_train, y_train)

# Test the model on the testing data
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

In [ ]:
# Load dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Fit custom Gaussian Naive Bayes
gnb_custom = naive_bayes.GaussianNaiveBayes()
gnb_custom.fit(X_train, y_train)
y_pred_custom = gnb_custom.predict(X_test)

# Fit sklearn Gaussian Naive Bayes for comparison
gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gnb_sklearn.predict(X_test)

# Compare results
custom_accuracy = np.mean(y_pred_custom == y_test)
sklearn_accuracy = np.mean(y_pred_sklearn == y_test)

print("Custom Gaussian Naive Bayes Accuracy:", custom_accuracy)
print("Sklearn Gaussian Naive Bayes Accuracy:", sklearn_accuracy)

In [ ]:
# Load dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Fit custom Gradient Boosting Classifier
gb_custom = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_custom.fit(X_train, y_train)
y_pred_custom = gb_custom.predict(X_test)

# Fit sklearn Gradient Boosting Classifier for comparison
gb_sklearn = SklearnGBC(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gb_sklearn.predict(X_test)

# Compare results
custom_accuracy = accuracy_score(y_test, y_pred_custom)
sklearn_accuracy = accuracy_score(y_test, y_pred_sklearn)

print("Custom Gradient Boosting Accuracy:", custom_accuracy)
print("Sklearn Gradient Boosting Accuracy:", sklearn_accuracy)

# Use the exi

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=42)
y = np.where(y == 0, -1, 1)  # AdaBoost in sklearn handles labels {0, 1}, converting to {-1, 1} for custom model

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the custom AdaBoost Classifier
custom_model = AdaBoostClassifier(n_clf=30)
custom_model.fit(X_train, y_train)
custom_preds = custom_model.predict(X_test)
custom_accuracy = accuracy_score(y_test, custom_preds)

# Initialize the sklearn AdaBoost Classifier
sklearn_model = SklearnAdaBoost(n_estimators=30, random_state=42, algorithm='SAMME')  # SAMME is the discrete boost algorithm similar to used in the custom implementation
sklearn_model.fit(X_train, y_train)
sklearn_preds = sklearn_model.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, sklearn_preds)

# Output the results
print("Custom AdaBoost Accuracy:", custom_accuracy)
print("Sklearn AdaBoost Accuracy:", sklearn_accuracy)

# For probability comparison, if implemented
custom_prob = custom_model.predict_proba(X_test[0].reshape(1, -1))
sklearn_prob = sklearn_model.predict_proba(X_test[0].reshape(1, -1))
print("Custom AdaBoost Sample Probabilities:", custom_prob)
print("Sklearn AdaBoost Sample Probabilities:", sklearn_prob)

In [ ]:
# Generating synthetic data
np.random.seed(0)
X, _ = make_classification(n_samples=100, n_features=5, random_state=42)
X = X.astype(float)
X[::4, 1] = np.nan  # Introduce missing values in the second column
X[::3, 3] = np.nan  # Introduce missing values in the fourth column

# Split the dataset to demonstrate usage
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Define custom and sklearn imputers
imputers = {
    "mean": (SimpleImputer(strategy="mean"), SklearnSimpleImputer(strategy="mean")),
    "median": (SimpleImputer(strategy="median"), SklearnSimpleImputer(strategy="median")),
    "mode": (SimpleImputer(strategy="mode"), SklearnSimpleImputer(strategy="most_frequent")),
    "knn": (KNNImputer(), SklearnKNNImputer()),
    # "iterative": (IterativeImputer(), SklearnIterativeImputer())
}

# Test each imputer
for name, (custom_imputer, sklearn_imputer) in imputers.items():
    custom_filled = custom_imputer.fit_transform(X_train)
    sklearn_filled = sklearn_imputer.fit_transform(X_train)

    # Compare using MSE to evaluate how close the custom implementation is to sklearn's
    mse = mean_squared_error(custom_filled, sklearn_filled)
    print(f"{name.capitalize()} Imputer MSE between custom and sklearn: {mse:.4f}")

In [6]:
def cross_val_score(model, X, y, scoring, cv=None):
    if cv is None:
        cv = KFold()

    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        score = scoring(y_test, predictions)
        scores.append(score)
    
    return np.array(scores)

In [7]:
# Load Iris dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Define the scoring function
def accuracy_scorer(y_true, y_pred):
    return np.mean(y_true == y_pred)    

# Use custom KFold and cross-validation
custom_cv = KFold(n_splits=3, shuffle=True, random_state=42)
model = DecisionTreeClassifier()
scores = cross_val_score(model, X, y, scoring=accuracy_scorer, cv=custom_cv)
print("Custom Implementation Scores:", scores)

# Compare with scikit-learn's implementation
sk_scores = sk_cross_val_score(DecisionTreeClassifier(), X, y, cv=3)
print("Scikit-Learn Implementation Scores:", sk_scores)

Custom Implementation Scores: [0.94 0.96 0.96]
Scikit-Learn Implementation Scores: [0.98 0.94 0.98]


In [8]:
# Load Iris dataset
data = datasets.load_iris()
X, y = data.data, data.target

# Define cross-validation setup
kfolds = KFold(n_splits=3, shuffle=True, random_state=42)

# Parameter grid
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 3, 4]
}

# Custom GridSearchCV implementation
custom_grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    scoring=accuracy_scorer,
    cv=kfolds
)

# Fit custom Grid Search
custom_grid_search.fit(X, y)
print("Custom GridSearchCV - Best Parameters:", custom_grid_search.best_params())
print("Custom GridSearchCV - Best Score:", custom_grid_search.best_score())

# Scikit-learn GridSearchCV
sklearn_grid_search = SklearnGridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3  # Number of folds, directly passed as an int to use default KFold in sklearn
)

# Fit scikit-learn Grid Search
sklearn_grid_search.fit(X, y)
print("Sklearn GridSearchCV - Best Parameters:", sklearn_grid_search.best_params_)
print("Sklearn GridSearchCV - Best Score:", sklearn_grid_search.best_score_)


Custom GridSearchCV - Best Parameters: {'max_depth': None, 'min_samples_split': 2}
Custom GridSearchCV - Best Score: 0.9533333333333333
Sklearn GridSearchCV - Best Parameters: {'max_depth': None, 'min_samples_split': 2}
Sklearn GridSearchCV - Best Score: 0.9733333333333333
